### Imports

In [292]:
from bs4 import BeautifulSoup
import requests
import json
import re
import string
from pymongo import MongoClient
from nltk.corpus import stopwords
import pyarabic.araby as araby
from pyarabic.number import ArNumbers
import pyarabic.number as arnb
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer as lem

### Loading the BeautifulSoup object

In [203]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
result = requests.get('https://www.hespress.com/economie', headers=headers)
doc = BeautifulSoup(result.text, 'html.parser')

### Retrieving the divs that contain the links to articles

In [204]:
main_div = doc.find_all(name='div',attrs={'class':'posts-categoy'})
posts = main_div[0].find_all(name='div',attrs={'class':'card-img-top'})
articals = []
for article in posts:
    # links
    link = article.find(name='a').get('href')
    # article titles
    title = article.find(name='a').get('title')
    articals.append({title : link})

### Storing links and titles in a json file

In [205]:
with open('hess_article_links.json', 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})
    json.dump(data, f, ensure_ascii=False, indent=2)

### Retrieving the divs that contain the links to articles

In [206]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        article_result = requests.get(article['link'], headers=headers)
        article_doc = BeautifulSoup(article_result.text, 'html.parser')
        article_content.append(article_doc.find_all(name='div',attrs={'class':'article-content'})[0].find_all(name='p'))

### Storing articles and their respective titles in a json file

In [217]:
with open('hess_article_content.json', 'w', encoding='utf-8') as f:
    contents = []
    for i,content in enumerate(article_content):
        content = ''
        for p in article_content[i]:
            content += p.text
        contents.append({'title': list(data[i].values())[0], 'content': content})
    json.dump(contents, f, ensure_ascii=False, indent=2)

### Uploading the two json files in a MongoDB database

In [218]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f1:
    links = json.load(f1)
with open('hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
client = MongoClient('mongodb://localhost:27017/')
db = client['NLP_lab1']
coll1 = db['hess_article_links']
coll2 = db['hess_article_content']
coll1.drop()
coll1.insert_many(links)
coll2.drop()
coll2.insert_many(art_content)
client.close()

### Loading the first article for the NLP pipeline

In [283]:
text = art_content[0]['content']
text.encode('utf-8')
print("Raw text: ",text)

Raw text:  يبدو أن مسلسل مساءلة ومحاسبة ليلى بنعلي، وزيرة الانتقال الطاقي والتنمية المستدامة، من قبل المعارضة في مجلس النواب مستمر، إذ أثارت المجموعة النيابية لحزب العدالة والتنمية موضوع اكتشافات الغاز المسال مرة أخرى وطالبت الحكومة بتوضيح المقتضيات القانونية المؤطرة لتدخل الفاعلين الأجانب في السوق الوطنية.ووجه عبد الله بووانو، رئيس المجموعة النيابية لحزب العدالة والتنمية، سؤالا كتابيا إلى الوزيرة أثار فيه غياب إطار تشريعي وقانوني خاص بقطاع الغاز المسال يؤطر تدخل الفاعلين الحاليين والمحتملين في سوق الغاز المسال بالمغرب، ودعا إلى الكشف عن “الإجراءات والآليات التي تعتمدها الرقابة على دخول شركات لا تحترم سيادة المغرب على هذا القطاع الحيوي”.وأفاد بووانو بأنه جرى اكتشاف الغاز الطبيعي في 49 بئرا، مع توقع قدرة إنتاجية مهمة بكل من تندرارة تقدر بـ300 مليون متر مكعب، وحقل انشوا تقارب 3 تريليون متر مكعب، وهو الأمر الذي ساهم في “جلب استثمارات أجنبية جديدة، وتوسيع مجال التنقيب والبحث عن الهيدروكاربورات أمام مجموعة من الشركات الأجنبية المتخصصة في التنقيب، إلى جانب إبرام اتفاقيات وعقود استغلال طويلة 

### Tokenization

In [284]:
tokens = araby.tokenize(text)
sent_tokens = araby.sentence_tokenize(text)
print("Word tokenized list: ", tokens)
print("Sentence tokenized list: ", sent_tokens)
print("Word tokenized list length: ", len(tokens))
print("Sentence tokenized list length: ", len(sent_tokens))

Word tokenized list:  ['يبدو', 'أن', 'مسلسل', 'مساءلة', 'ومحاسبة', 'ليلى', 'بنعلي', '،', 'وزيرة', 'الانتقال', 'الطاقي', 'والتنمية', 'المستدامة', '،', 'من', 'قبل', 'المعارضة', 'في', 'مجلس', 'النواب', 'مستمر', '،', 'إذ', 'أثارت', 'المجموعة', 'النيابية', 'لحزب', 'العدالة', 'والتنمية', 'موضوع', 'اكتشافات', 'الغاز', 'المسال', 'مرة', 'أخرى', 'وطالبت', 'الحكومة', 'بتوضيح', 'المقتضيات', 'القانونية', 'المؤطرة', 'لتدخل', 'الفاعلين', 'الأجانب', 'في', 'السوق', 'الوطنية', '.', 'ووجه', 'عبد', 'الله', 'بووانو', '،', 'رئيس', 'المجموعة', 'النيابية', 'لحزب', 'العدالة', 'والتنمية', '،', 'سؤالا', 'كتابيا', 'إلى', 'الوزيرة', 'أثار', 'فيه', 'غياب', 'إطار', 'تشريعي', 'وقانوني', 'خاص', 'بقطاع', 'الغاز', 'المسال', 'يؤطر', 'تدخل', 'الفاعلين', 'الحاليين', 'والمحتملين', 'في', 'سوق', 'الغاز', 'المسال', 'بالمغرب', '،', 'ودعا', 'إلى', 'الكشف', 'عن', '“', 'الإجراءات', 'والآليات', 'التي', 'تعتمدها', 'الرقابة', 'على', 'دخول', 'شركات', 'لا', 'تحترم', 'سيادة', 'المغرب', 'على', 'هذا', 'القطاع', 'الحيوي', '”.', 'وأفاد', 'ب

### Removing punctuation

In [285]:
ar_punct = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”،.”…“–ـ”.'''
en_punct = string.punctuation
punct_lst = ar_punct + en_punct

In [286]:
tokens = [token for token in tokens if token not in punct_lst]
sent_tokens = [sent.translate(str.maketrans('', '', punct_lst)) for sent in sent_tokens]
print("Tokenized text without punctuation: ",tokens)
print("Sentence tokenized text without punctuation: ",sent_tokens)
print("Length of tokenized text without punctuation: ", len(tokens))
print("Length of sentence tokenized text without punctuation: ", len(sent_tokens))

Tokenized text without punctuation:  ['يبدو', 'أن', 'مسلسل', 'مساءلة', 'ومحاسبة', 'ليلى', 'بنعلي', 'وزيرة', 'الانتقال', 'الطاقي', 'والتنمية', 'المستدامة', 'من', 'قبل', 'المعارضة', 'في', 'مجلس', 'النواب', 'مستمر', 'إذ', 'أثارت', 'المجموعة', 'النيابية', 'لحزب', 'العدالة', 'والتنمية', 'موضوع', 'اكتشافات', 'الغاز', 'المسال', 'مرة', 'أخرى', 'وطالبت', 'الحكومة', 'بتوضيح', 'المقتضيات', 'القانونية', 'المؤطرة', 'لتدخل', 'الفاعلين', 'الأجانب', 'في', 'السوق', 'الوطنية', 'ووجه', 'عبد', 'الله', 'بووانو', 'رئيس', 'المجموعة', 'النيابية', 'لحزب', 'العدالة', 'والتنمية', 'سؤالا', 'كتابيا', 'إلى', 'الوزيرة', 'أثار', 'فيه', 'غياب', 'إطار', 'تشريعي', 'وقانوني', 'خاص', 'بقطاع', 'الغاز', 'المسال', 'يؤطر', 'تدخل', 'الفاعلين', 'الحاليين', 'والمحتملين', 'في', 'سوق', 'الغاز', 'المسال', 'بالمغرب', 'ودعا', 'إلى', 'الكشف', 'عن', 'الإجراءات', 'والآليات', 'التي', 'تعتمدها', 'الرقابة', 'على', 'دخول', 'شركات', 'لا', 'تحترم', 'سيادة', 'المغرب', 'على', 'هذا', 'القطاع', 'الحيوي', 'وأفاد', 'بووانو', 'بأنه', 'جرى', 'اكتشاف'

### Removing stopwords

In [287]:
stop_words = set(stopwords.words('arabic'))
print("Stop words: ",stop_words)
tokens = [word for word in tokens if word not in stop_words]
sent_tokens = [''.join([x for x in re.split(r'(\W+)', sent) if x not in stop_words]) for sent in sent_tokens]

print("Tokenized text without stopwords: ",tokens)
print("Sentence tokenized text without stopwords: ",sent_tokens)
print("Length of tokenized text without stopwords: ", len(tokens))
print("Length of sentence tokenized text without stopwords: ", len(sent_tokens))

Stop words:  {'أخذ', 'أنبأ', 'أين', 'كثيرا', 'آناء', 'فإذا', 'ليستا', 'ق', 'تلكم', 'حمدا', 'ثلاثين', 'اللتين', 'أنتما', 'بعض', 'ى', 'جنيه', 'يفعلون', 'ثمانون', 'طالما', 'متى', 'لعلَّ', 'سبعمائة', 'ممن', 'سادس', 'انقلب', 'لئن', 'أم', 'لستم', 'فاء', 'أمامك', 'اثنين', 'كلاهما', 'حتى', 'لكما', 'ثلاثمائة', 'كسا', 'أنتم', 'ثمانية', 'لست', 'خبَّر', 'بهم', 'مساء', 'حقا', 'خلف', 'فوق', 'شتان', 'ساء', 'خامس', 'اللتان', 'إياهن', 'فلا', 'نا', 'غ', 'لهما', 'أفريل', 'هللة', 'أيلول', 'ديسمبر', 'بما', 'ثمّ', 'ضاد', 'تانِ', 'ياء', 'شيكل', 'ذانك', 'من', 'ولا', 'صبرا', 'تحوّل', 'ذلكن', 'شباط', 'وما', 'بعد', 'أولالك', 'أوت', 'هنا', 'استحال', 'أجمع', 'حمٌ', 'فإن', 'اثنا', 'أنتن', 'أنّى', 'كِخ', 'شبه', 'رزق', 'مازال', 'صاد', 'إلا', 'س', 'ستمئة', 'اللواتي', 'سبعمئة', 'جوان', 'لكم', 'اثني', 'كأنما', 'هَذِي', 'ئ', 'ذ', 'باء', 'بضع', 'جويلية', 'لعمر', 'إذ', 'ذو', 'أ', 'لها', 'أكثر', 'إليك', 'هيت', 'شَتَّانَ', 'أولئك', 'ذي', 'يوليو', 'ذواتي', 'قبل', 'راء', 'خمسمئة', 'هذا', 'ثان', 'طاق', 'هَذَيْنِ', 'إن', 'أسكن',

### Convert numbers to words

In [288]:
arnum = ArNumbers()
num_tokens = []
for i,token in enumerate(tokens):
    if tokens[i].isdigit():
        tokens[i] = arnum.int2str(tokens[i])
        num_tokens.append(tokens[i])
print("Number to string: ", tokens)
print("Numerical tokens: ", num_tokens)

Number to string:  ['يبدو', 'مسلسل', 'مساءلة', 'ومحاسبة', 'ليلى', 'بنعلي', 'وزيرة', 'الانتقال', 'الطاقي', 'والتنمية', 'المستدامة', 'المعارضة', 'مجلس', 'النواب', 'مستمر', 'أثارت', 'المجموعة', 'النيابية', 'لحزب', 'العدالة', 'والتنمية', 'موضوع', 'اكتشافات', 'الغاز', 'المسال', 'مرة', 'أخرى', 'وطالبت', 'الحكومة', 'بتوضيح', 'المقتضيات', 'القانونية', 'المؤطرة', 'لتدخل', 'الفاعلين', 'الأجانب', 'السوق', 'الوطنية', 'ووجه', 'عبد', 'الله', 'بووانو', 'رئيس', 'المجموعة', 'النيابية', 'لحزب', 'العدالة', 'والتنمية', 'سؤالا', 'كتابيا', 'الوزيرة', 'أثار', 'غياب', 'إطار', 'تشريعي', 'وقانوني', 'خاص', 'بقطاع', 'الغاز', 'المسال', 'يؤطر', 'تدخل', 'الفاعلين', 'الحاليين', 'والمحتملين', 'سوق', 'الغاز', 'المسال', 'بالمغرب', 'ودعا', 'الكشف', 'الإجراءات', 'والآليات', 'تعتمدها', 'الرقابة', 'دخول', 'شركات', 'تحترم', 'سيادة', 'المغرب', 'القطاع', 'الحيوي', 'وأفاد', 'بووانو', 'بأنه', 'جرى', 'اكتشاف', 'الغاز', 'الطبيعي', 'تسع و أربعون', 'بئرا', 'توقع', 'قدرة', 'إنتاجية', 'مهمة', 'بكل', 'تندرارة', 'تقدر', 'بـ300', 'مليون'

### Normalization

In [289]:
ArListem = ArabicLightStemmer()
# removing numerical tokens
for i in num_tokens:
    tokens.remove(i)
tokens = [ArListem.normalize(token) for token in tokens]
print("Normalized token list: ", tokens)

Normalized token list:  ['يبدو', 'مسلسل', 'مساءله', 'ومحاسبه', 'ليلي', 'بنعلي', 'وزيره', 'الانتقال', 'الطاقي', 'والتنميه', 'المستدامه', 'المعارضه', 'مجلس', 'النواب', 'مستمر', 'اثارت', 'المجموعه', 'النيابيه', 'لحزب', 'العداله', 'والتنميه', 'موضوع', 'اكتشافات', 'الغاز', 'المسال', 'مره', 'اخري', 'وطالبت', 'الحكومه', 'بتوضيح', 'المقتضيات', 'القانونيه', 'المءطره', 'لتدخل', 'الفاعلين', 'الاجانب', 'السوق', 'الوطنيه', 'ووجه', 'عبد', 'الله', 'بووانو', 'رءيس', 'المجموعه', 'النيابيه', 'لحزب', 'العداله', 'والتنميه', 'سءالا', 'كتابيا', 'الوزيره', 'اثار', 'غياب', 'اطار', 'تشريعي', 'وقانوني', 'خاص', 'بقطاع', 'الغاز', 'المسال', 'يءطر', 'تدخل', 'الفاعلين', 'الحاليين', 'والمحتملين', 'سوق', 'الغاز', 'المسال', 'بالمغرب', 'ودعا', 'الكشف', 'الاجراءات', 'والاليات', 'تعتمدها', 'الرقابه', 'دخول', 'شركات', 'تحترم', 'سياده', 'المغرب', 'القطاع', 'الحيوي', 'وافاد', 'بووانو', 'بانه', 'جري', 'اكتشاف', 'الغاز', 'الطبيعي', 'بءرا', 'توقع', 'قدره', 'انتاجيه', 'مهمه', 'بكل', 'تندراره', 'تقدر', 'ب300', 'مليون', 'متر', 'مك

### Stemming

In [295]:
stemmed_tokens = [ArListem.light_stem(token) for token in tokens]
print("Stemmed token list: ", stemmed_tokens)

Stemmed token list:  ['بدو', 'مسلسل', 'مساءل', 'محاسب', 'لي', 'نعل', 'زير', 'انتقال', 'طاق', 'تنميه', 'مستدامه', 'معارضه', 'مجلس', 'نواب', 'مستمر', 'اثار', 'مجموعه', 'نيابيه', 'حزب', 'عداله', 'تنميه', 'موضوع', 'اكتشاف', 'غاز', 'مسال', 'مر', 'خر', 'طالب', 'حكومه', 'توضيح', 'مقتض', 'قانونيه', 'مءطره', 'دخل', 'فاعل', 'اجانب', 'سوق', 'وطنيه', 'وج', 'عبد', 'له', 'ووانو', 'رءيس', 'مجموعه', 'نيابيه', 'حزب', 'عداله', 'تنميه', 'سءال', 'تابيا', 'وزيره', 'ثار', 'غياب', 'طار', 'شريع', 'قان', 'خاص', 'قطاع', 'غاز', 'مسال', 'ءطر', 'دخل', 'فاعل', 'حالي', 'محتمل', 'سوق', 'غاز', 'مسال', 'مغرب', 'دع', 'كشف', 'اجراء', 'ال', 'عتمد', 'رقابه', 'دخول', 'شرك', 'حترم', 'سياد', 'مغرب', 'قطاع', 'حيو', 'فاد', 'ووانو', 'ان', 'جر', 'كتشاف', 'غاز', 'طبيع', 'ءرا', 'وقع', 'قدر', 'نتاج', 'مهم', 'كل', 'تندرار', 'قدر', '300', 'ملي', 'متر', 'مكعب', 'حقل', 'نش', 'قارب', 'ريلي', 'متر', 'مكعب', 'امر', 'سا', 'جلب', 'استثمار', 'جنب', 'جديد', 'وسيع', 'مجال', 'تنقيب', 'بحث', 'الهيدروكاربورات', 'مجموع', 'شرك', 'اجنبيه', 'متخصصه', 

### Lemmatization

In [296]:
lemmer = lem.Lemmatizer()
lemmatized_tokens = [lemmer.lemmatize(token) for token in tokens]
print("Lemmatized token list: ", lemmatized_tokens)

Lemmatized token list:  ['بدا', 'مسلسل', 'مساءله', 'محاسب', 'ليل', 'نعل', 'زير', 'انتقال', 'طاق', 'والتنميه', 'المستدامه', 'المعارضه', 'مجلس', 'نواب', 'مستمر', 'اثارت', 'المجموعه', 'النيابيه', 'حزب', 'العداله', 'والتنميه', 'موضوع', 'اكتشاف', 'غاز', 'مسال', 'مر', 'اخري', 'طالب', 'الحكومه', 'توضيح', 'مقتضى', 'القانونيه', 'المءطره', 'تدخل', 'فاعل', 'الاجانب', 'سوق', 'الوطنيه', 'وجه', 'عبد', 'الله', 'بووانو', 'رءيس', 'المجموعه', 'النيابيه', 'حزب', 'العداله', 'والتنميه', 'سءالا', 'كتاب', 'الوزيره', 'اثار', 'غياب', 'اطار', 'تشريع', 'قانون', 'خاص', 'قطاع', 'غاز', 'مسال', 'يءطر', 'تدخل', 'فاعل', 'حالي', 'محتمل', 'سوق', 'غاز', 'مسال', 'مغرب', 'ودع', 'كشف', 'الاجراءات', 'ال', 'اعتمد', 'الرقابه', 'دخول', 'شركة', 'احترم', 'سياده', 'مغرب', 'قطاع', 'حيوي', 'وافاد', 'بووانو', 'بان', 'جرة', 'اكتشاف', 'غاز', 'طبيعة', 'بءرا', 'توقع', 'قدر', 'انتاجيه', 'مهم', 'كل', 'تندراره', 'تقدر', 'ب300', 'مليون', 'متر', 'مكعب', 'حقل', 'نشي', 'تقارب', 'تريليون', 'متر', 'مكعب', 'الامر', 'ساهم', 'جلب', 'استثمار', 'جنب',

### Stemming vs lemmatization comparison

In [297]:
print("Original token list: ", tokens)
print("Stemmed token list: ", stemmed_tokens)
print("Lemmatized token list: ", lemmatized_tokens)

Original token list:  ['يبدو', 'مسلسل', 'مساءله', 'ومحاسبه', 'ليلي', 'بنعلي', 'وزيره', 'الانتقال', 'الطاقي', 'والتنميه', 'المستدامه', 'المعارضه', 'مجلس', 'النواب', 'مستمر', 'اثارت', 'المجموعه', 'النيابيه', 'لحزب', 'العداله', 'والتنميه', 'موضوع', 'اكتشافات', 'الغاز', 'المسال', 'مره', 'اخري', 'وطالبت', 'الحكومه', 'بتوضيح', 'المقتضيات', 'القانونيه', 'المءطره', 'لتدخل', 'الفاعلين', 'الاجانب', 'السوق', 'الوطنيه', 'ووجه', 'عبد', 'الله', 'بووانو', 'رءيس', 'المجموعه', 'النيابيه', 'لحزب', 'العداله', 'والتنميه', 'سءالا', 'كتابيا', 'الوزيره', 'اثار', 'غياب', 'اطار', 'تشريعي', 'وقانوني', 'خاص', 'بقطاع', 'الغاز', 'المسال', 'يءطر', 'تدخل', 'الفاعلين', 'الحاليين', 'والمحتملين', 'سوق', 'الغاز', 'المسال', 'بالمغرب', 'ودعا', 'الكشف', 'الاجراءات', 'والاليات', 'تعتمدها', 'الرقابه', 'دخول', 'شركات', 'تحترم', 'سياده', 'المغرب', 'القطاع', 'الحيوي', 'وافاد', 'بووانو', 'بانه', 'جري', 'اكتشاف', 'الغاز', 'الطبيعي', 'بءرا', 'توقع', 'قدره', 'انتاجيه', 'مهمه', 'بكل', 'تندراره', 'تقدر', 'ب300', 'مليون', 'متر', 'مكعب

## References
- Zerrouki, T., (2023). PyArabic: A Python package for Arabic text. Journal of Open Source Software, 8(84), 4886, https://doi.org/10.21105/joss.04886
- Alkhatib, R. M., Zerrouki, T., Shquier, M. M. A., & Balla, A. (2023). Tashaphyne0.4: A new arabic light stemmer based on rhyzome modeling approach. Information Retrieval Journa, 26(14). doi: https://doi.org/10.1007/s10791-023-09429-y
- T. Zerrouki, Qalsadi, Arabic mophological analyzer Library for python.,  https://pypi.python.org/pypi/qalsadi/
- Hegazi, M. O., Al-Dossari, Y., Al-Yahy, A., Al-Sumari, A., & Hilal, A. (2021). Preprocessing Arabic text on social media. Heliyon, 7(2), e06191. doi:10.1016/j.heliyon.2021.e06191 
- Sawalha, M., Atwell, E., & Abushariah, M. A. M. (2013). SALMA: Standard Arabic Language Morphological Analysis. 2013 1st International Conference on Communications, Signal Processing, and Their Applications (ICCSPA). doi:10.1109/iccspa.2013.6487311 